# GDC April 2021 Webinar: Using the GDC API

### Monday, April 26, 2021<br>2:00 PM - 3:00 PM (EST)<br>Bill Wysocki, Director of User Services <br>University of Chicago

## API User's Guide and Other Helpful Links

[GDC API User's Guide](https://docs.gdc.cancer.gov/API/Users_Guide/Getting_Started/)

[GDC Support Website](https://gdc.cancer.gov/support)

support@nci-gdc.datacommons.io - GDC Helpdesk E-mail

[Requests Python Package User's Guide](https://2.python-requests.org/en/master/)

[Python Documentation Website](https://www.python.org)

# Notebook Overview 


### About this notebook

- This notebook serves to be a resource for GDC users to familiarize themselves with the GDC API endpoints and capablities and allow users to edit and create custom queries "in place" with template functions, as well as perform other data analyses and visualizations within the Jupyter Notebook interface
- Jupyter Notebook documnetation can be found at https://jupyter.org 
- Commands and functions in this notebook will rely on the following Python packages:
    - `requests` - if not already installed on your system, can use `pip install requests` from command line or in new notebook code cell
    - `json` - part of Python standard library, should already be installed on system
    - `urllib` - part of Python standard library, should already be installed on system

In [3]:
#import packages to use in this notebook

import requests
import json
import urllib

### Table of Contents

- [GDC API Overview](#api_overview')
- [How to Search and Retrieve Data with GDC API](#search_retrieve)
- [How to Download Files with GDC API](#download)
- [How to Perform BAM Slicing with GDC API](#bam_slice)
- [How to Submit Data with GDC API](#submit)


# <a id='api_overview'>GDC API Overview</a>


- The GDC Application Programming Interface (API) is the external facing REpresentational State Transfer (REST) interface for the GDC, which supports user interactions with the GDC Submission and Data Portals, as well as provides developers with a programmatic interface to query and download GDC data, metadata and annotations and submit data to the GDC.
- The GDC API uses JSON as its communication format, and uses standard HTTP methods (GET, PUT, POST and DELETE)
- The [GDC Data Transfer Tool](https://gdc.cancer.gov/access-data/gdc-data-transfer-tool) client also relies on the GDC API for user authentication, reading manifests, and for download and upload features


### GDC API Format

- GDC API URL is: https://api.gdc.cancer.gov/
- GDC API format for non-submission use is: <b>API_URL + ENDPOINT + QUERY_PARAMETERS</b>
    - In order to utilize the GDC API, calls to specific API endpoints for a given task need to be made
    - Query parameters can be included, such as <b>filters</b> to search and distill results, and <b>fields</b> or types of data to return
    - Formatting parameters can be specified such as <b>format</b> and <b>pretty</b>
    - Examples of endpoints and possible query parameters:
- The format for using the GDC API Submission enpoint = https://api.gdc.cancer.gov/<b>program_name/project_code</b>, i.e. https://api.gdc.cancer.gov/submission/TCGA/LUAD or https://api.gdc.cancer.gov/submission/CPTAC/3 

# <a id='search_retrieve'>How to Search and Retrieve Data with GDC API</a>

### Overview

### Endpoints

There are two 'types' of endpoints that can be used to search and retrieve data:


[GDC Search and Retrieval Endpoints](https://docs.gdc.cancer.gov/API/Users_Guide/Search_and_Retrieval/#endpoints) - includes endpoints that cover `project`, `file` and `case` information, including clinical and biospecimen metadata, as well as file version and history

[GDC Analysis Endpoints](https://docs.gdc.cancer.gov/API/Users_Guide/Data_Analysis/) - endpoints that are used by the GDC data analysis, visualization and exploration (DAVE) tools in the Exploration tab of the GDC Data Portal to access indexed data including gene, mutation, copy number variation and survival data. 


### Steps

1. Specify and encode `filters`
2. Specify `fields` to be returned
3. Specify additional parameters (`size`, `format` of results etc.)
3. Build query url
4. Submit query and save response text to file

*specifying parameters are optional; not specifying filters will return all results at endpoint, and not specifying fields will return all fields at endpoint, while other parameters will be set to default value (i.e. size = 10, format = JSON)

### Template queryBuilder() function

- Search and Retrieval requests can be built as a url with the api and endpoint and pther parameters specified
- Users can edit a template function to build url request for querying data in GDC API to include other parameters, such as `facets`, `expand`, `from` (pagination) and `sort`: 
https://docs.gdc.cancer.gov/API/Users_Guide/Search_and_Retrieval/#request-parameters
- To specify default parameters, users can simply input two quotation marks, i.e. `''`, for a given varible to the queryBuilder() function

In [16]:
#format is specified as 'frmat' in function as format is an already declared object in python [the format() function]

def queryBuilder(endpoint, filters, fields, size, frmat):
    api_url = 'https://api.gdc.cancer.gov/'
    
    if frmat.lower() == 'json':
        request_query = api_url + endpoint + '?filters=' + filters + '&fields=' + fields + '&size=' + size + '&format=' + frmat + '&pretty=true'
    else:
        request_query = api_url + endpoint + '?filters=' + filters + '&fields=' + fields + '&size=' + size + '&format=' + frmat
    return request_query

### Templates for query search parameters (<font color="red">filters</font>)

- describe filters (operators, fields and values)
- need to be in JSON format that then will need to be percent encoded to be sent in url request
- will use `urllib` package for formatting
- link to [operators](https://docs.gdc.cancer.gov/API/Users_Guide/Search_and_Retrieval/#filters-specifying-the-query)
- link to filters/fields 
- specifying no filters will return all entities for a given endpoint
- need to use filters/fields of that endpoint
- several examples below to edit 

In [ ]:
#one filter applied to endpoint

#one filter 
one_filter = {
            "op":"=",
            "content":{
                "field": "cases.project.project_id", 
                "value": "TCGA-BRCA"
    }
}

In [ ]:
#combination of two filters applied to endpoint, i.e. (x AND/OR y) must be met

combination_two = {
    "op" : "and",
    "content":[{
        "op":"=",
         "content":{
              "field": "cases.project.project_id", 
                "value": "TCGA-BRCA"
            }
        }, 
        {
            "op":"=", 
            "content":{
                "field":"cases.disease_type",
                "value": "ductal and lobular neoplasms"
            }
        }
    ]
}

In [ ]:
#combination of three filters applied to endpoint, i.e. (x AND/OR y AND/OR z) must be met

combination_three = {
    "op" : "and",
    "content":[{
        "op":"=",
         "content":{
              "field": "cases.project.project_id", 
                "value": "TCGA-BRCA"
            }
        }, 
        {
            "op":"=", 
            "content":{
                "field":"cases.disease_type",
                "value": "ductal and lobular neoplasms"
            }
        },
        {
            "op":">", 
            "content":{
                "field":"diagnoses.age_at_diagnosis",
                "value": "15000"
            }
        }
        
    ]
}

In [ ]:
#complex combination of three filters applied to endpoint, i.e. (x AND/OR [y AND/OR z]) must be met

combination_three_2 = {
    "op": "and",
    "content": [{
            "op": "=",
            "content": {
                "field": "cases.project.project_id",
                "value": "TCGA-BRCA"
            }
        },
        {
            "op": "or",
            "content": [{
                    "op": "=",
                    "content": {
                        "field": "cases.disease_type",
                        "value": "cystic, mucinous and serious neoplasms"
                    }
                },
                {
                    "op": "=",
                    "content": {
                        "field": "cases.disease_type",
                        "value": "ductal and lobular neoplasms"
                    }
                }
            ]
        }
    ]
}

### Template commands for formatting search query parameters

In [ ]:
#percent encoding of filters
json_string=str(json.dumps(one_filter))
example_filter = urllib.parse.quote(json_string.encode('utf-8'))

### Template for formatting <font color="red">fields</font> to be returned by query

- all fields = do not specify?
- comma delimited list of fields to be returned
- specifying no fields will return all available fields for entities that match filters in endpoint

In [ ]:
#specify fields to be returned
example_fields = ",".join([
    "submitter_id",
    "disease_type",
    "samples.submitter_id",
    "samples.sample_type", 
    "samples.tissue_type",
    "diagnoses.age_at_diagnosis"
])

### Template API `GET` Request 

In [ ]:
#build API query: queryBuilder(endpoint, filters, fields, size, frmat)

#to specify no filters and/or no fields to return, replace variable with <''>

template_request = queryBuilder('cases', example_filter, example_fields, '11315', "json")

template_request

##### Note: You can also copy and paste formatted request URL into browser url bar to  return results in browser

In [ ]:
#send request
result = requests.get(template_request)

#write request results to file, edit file name and type 
with open("/Users/catherineausland/Desktop/ffpe.json", "w+") as output: 
    output.write(result.text)
output.close()

### Example 1: Retrieve case barcode, sample type and primary diagnosis data for DNA-seq files in TCGA-BRCA project

- Retrieve whether BAM files are for normal or tumor samples, as well as what disease cases were diagnosed as, in the TCGA-BRCA project
- Use 'files' endpoint, as this endpoint contains metadata related to files in the GDC (such as experimental strategy and data category)
- Need to filter down to files that are of the data category "sequencing reads" and experimental strategy type "WXS" (whole exome) to filter out other categories (like copy number variation, gene expression) and other experimental stragies (like RNA-Seq). 

In [30]:
#step 1: specify and encode filters

filters = {
    "op" : "and",
    "content":[{
        "op":"=",
         "content":{
              "field": "cases.project.project_id", 
                "value": "TCGA-BRCA"
            }
        }, 
        {
            "op":"=", 
            "content":{
                "field":"files.data_category",
                "value": "sequencing reads"
            }
        },
        {
            "op":"=", 
            "content":{
                "field":"files.experimental_strategy",
                "value": "WXS"
            }
        },
        {
            "op":"=", 
            "content":{
                "field":"files.data_format",
                "value": "BAM"
            }
        }
        
    ]
}

json_string=str(json.dumps(filters))
filters_format = urllib.parse.quote(json_string.encode('utf-8'))

#step 2: specify fields to be returned
fields = ",".join([
    "cases.submitter_id",
    "file_name",
    "cases.samples.sample_type",
    "cases.diagnoses.primary_diagnosis"
])

#step 3+4: build query url with 'files' endpoint, specify size=1 and format=tsv
brca_request = queryBuilder('files', filters_format, fields, '1', "tsv")

#step 5: send request
brca_result = requests.get(brca_request)

print(brca_result.text)
brca_request

cases.0.diagnoses.0.primary_diagnosis	cases.0.samples.0.sample_type	cases.0.submitter_id	file_name	id
Infiltrating duct carcinoma, NOS	Blood Derived Normal	TCGA-AR-A0TQ	TCGA-AR-A0TQ-10A-01D-A099-09_IlluminaGA-DNASeq_exome_gdc_realn.bam	7e089ffc-a3c6-4182-b14a-b0c75c829af2



'https://api.gdc.cancer.gov/files?filters=%7B%22op%22%3A%20%22and%22%2C%20%22content%22%3A%20%5B%7B%22op%22%3A%20%22%3D%22%2C%20%22content%22%3A%20%7B%22field%22%3A%20%22cases.project.project_id%22%2C%20%22value%22%3A%20%22TCGA-BRCA%22%7D%7D%2C%20%7B%22op%22%3A%20%22%3D%22%2C%20%22content%22%3A%20%7B%22field%22%3A%20%22files.data_category%22%2C%20%22value%22%3A%20%22sequencing%20reads%22%7D%7D%2C%20%7B%22op%22%3A%20%22%3D%22%2C%20%22content%22%3A%20%7B%22field%22%3A%20%22files.experimental_strategy%22%2C%20%22value%22%3A%20%22WXS%22%7D%7D%2C%20%7B%22op%22%3A%20%22%3D%22%2C%20%22content%22%3A%20%7B%22field%22%3A%20%22files.data_format%22%2C%20%22value%22%3A%20%22BAM%22%7D%7D%5D%7D&fields=cases.submitter_id,file_name,cases.samples.sample_type,cases.diagnoses.primary_diagnosis&size=1&format=tsv'

### Example 2: Retrieve FFPE data for samples and portions for TCGA projects

- Retrieve whether case samples and portions taken from cases in TCGA projects were Formalin-Fixed Paraffin-Embedded (FFPE) specimens or not
- Will use the 'cases' endpoint, as this endpoint contains biospecien and clinical information related to cases and samples in the GDC

In [29]:
#step 1: specify and encode filters
filters = {
            "op":"=",
            "content":{
                "field": "cases.project.program.name", 
                "value": "TCGA"
    }
}

json_string=str(json.dumps(filters))
filters_format = urllib.parse.quote(json_string.encode('utf-8'))

#step 2: specify fields to be returned
fields = ",".join([
    "submitter_id",
    "samples.submitter_id",
    "samples.is_ffpe",
    "samples.portions.submitter_id",
    "samples.portions.is_ffpe", 
])

#step 3+4: build query url with 'cases' endpoint, specify size=1 and format=json
ffpe_request = queryBuilder('cases', filters_format, fields, '1', "json")

#step 5: send request
ffpe_result = requests.get(ffpe_request)

print(ffpe_result.text)
ffpe_request

{
  "data": {
    "hits": [
      {
        "id": "375436b3-66ac-4d5e-b495-18a96d812a69", 
        "submitter_id": "TCGA-F5-6810", 
        "samples": [
          {
            "submitter_id": "TCGA-F5-6810-01Z", 
            "is_ffpe": true
          }, 
          {
            "submitter_id": "TCGA-F5-6810-10A", 
            "is_ffpe": false, 
            "portions": [
              {
                "submitter_id": "TCGA-F5-6810-10A-01", 
                "is_ffpe": false
              }
            ]
          }, 
          {
            "submitter_id": "TCGA-F5-6810-01A", 
            "is_ffpe": false, 
            "portions": [
              {
                "submitter_id": "TCGA-F5-6810-01A-11", 
                "is_ffpe": false
              }, 
              {
                "submitter_id": "TCGA-F5-6810-01A-13-1935-20", 
                "is_ffpe": false
              }
            ]
          }
        ]
      }
    ], 
    "pagination": {
      "count": 1, 
      "total": 1

'https://api.gdc.cancer.gov/cases?filters=%7B%22op%22%3A%20%22%3D%22%2C%20%22content%22%3A%20%7B%22field%22%3A%20%22cases.project.program.name%22%2C%20%22value%22%3A%20%22TCGA%22%7D%7D&fields=submitter_id,samples.submitter_id,samples.is_ffpe,samples.portions.submitter_id,samples.portions.is_ffpe&size=1&format=json&pretty=true'

### Example 3: Age at Diagnosis, Days to Death after Diagnosis, Vital Status and other clinical data for cases in TCGA-KIRC project

- Retrieve 
- Will only print results for `demographic.days_to_death` if case is dead

In [33]:
#step 1: specify and encode filters
filters = {
            "op":"=",
            "content":{
                "field": "cases.project.project_id", 
                "value": "TCGA-KIRC"
    }
}

json_string=str(json.dumps(filters))
filters_format = urllib.parse.quote(json_string.encode('utf-8'))

#step 2: specify fields to be returned
fields = ",".join([
    "submitter_id",
    "diagnoses.age_at_diagnosis",
    "demographic.days_to_death",
    "demographic.vital_status", 
    "demographic.ethnicity",
    "demographic.race",
    "demographic.gender"
])

#step 3+4: build query url with 'cases' endpoint, specify size=1 and format=json
age_request = queryBuilder('cases', filters_format, fields, '2', "tsv")

#step 5: send request
age_result = requests.get(age_request)

print(age_result.text)
age_request

demographic.days_to_death	demographic.ethnicity	demographic.gender	demographic.race	demographic.vital_status	diagnoses.0.age_at_diagnosis	id	submitter_id
	not hispanic or latino	male	white	Alive	15197	24c1cf70-bd67-431e-a623-d20f8d3f52b2	TCGA-CJ-4881
866	not hispanic or latino	male	white	Dead	21317	12adefc4-c9fd-46d3-904b-8fc52d5f1913	TCGA-B0-4696



'https://api.gdc.cancer.gov/cases?filters=%7B%22op%22%3A%20%22%3D%22%2C%20%22content%22%3A%20%7B%22field%22%3A%20%22cases.project.project_id%22%2C%20%22value%22%3A%20%22TCGA-KIRC%22%7D%7D&fields=submitter_id,diagnoses.age_at_diagnosis,demographic.days_to_death,demographic.vital_status,demographic.ethnicity,demographic.race,demographic.gender&size=2&format=tsv'

### Example 4: Genes information

- Query gene position and symbol for genes on chromosome 8

In [38]:
#step 1: specify and encode filters
filters = {
            "op":"=",
            "content":{
                "field": "gene_chromosome", 
                "value": "8"
    }
}

json_string=str(json.dumps(filters))
filters_format = urllib.parse.quote(json_string.encode('utf-8'))

#step 2: specify fields to be returned
fields = ",".join([
    "id",
    "symbol",
    "gene_start",
    "gene_end"
])

#step 3+4: build query url with 'cases' endpoint, specify size=1 and format=json
genes_request = queryBuilder('genes', filters_format, fields, '10', "tsv")

#step 5: send request
genes_result = requests.get(genes_request)

print(genes_result.text)
genes_request

gene_end	gene_start	id	symbol
7823889	7821966	ENSG00000186562	DEFB105A
108787615	108606850	ENSG00000164841	TMEM74
144428563	144423601	ENSG00000160948	VPS28
40897833	40530590	ENSG00000165061	ZMAT4
143384220	143368855	ENSG00000158106	RHPN1
108083648	107899316	ENSG00000147655	RSPO2
143609773	143603913	ENSG00000104524	PYCRL
101372707	101361794	ENSG00000228224	NACAP1
20183206	20144855	ENSG00000036565	SLC18A1
18084985	18056425	ENSG00000104763	ASAH1



'https://api.gdc.cancer.gov/genes?filters=%7B%22op%22%3A%20%22%3D%22%2C%20%22content%22%3A%20%7B%22field%22%3A%20%22gene_chromosome%22%2C%20%22value%22%3A%20%228%22%7D%7D&fields=id,symbol,gene_start,gene_end&size=10&format=tsv'

In [55]:
#Can use gene_id to also query individual information about the gene in question from genes endpoint as well
#by appending the gene_id at the end of the 'genes' endpoint and specifying parameters

individual_gene_request = requests.get('https://api.gdc.cancer.gov/genes/ENSG00000160948?pretty=true')

print(individual_gene_request.text)

{
  "data": {
    "biotype": "protein_coding", 
    "symbol": "VPS28", 
    "canonical_transcript_length_genomic": 4963, 
    "cytoband": [
      "8q24.3"
    ], 
    "canonical_transcript_length": 1097, 
    "synonyms": [], 
    "description": "This gene encodes a protein subunit of the ESCRT-I complex (endosomal complexes required for transport), which functions in the transport and sorting of proteins into subcellular vesicles. This complex can also be hijacked to facilitate the budding of enveloped viruses from the cell membrane. Alternative splicing results in multiple transcript variants encoding different isoforms. [provided by RefSeq, Jul 2013]", 
    "canonical_transcript_length_cds": 699, 
    "gene_chromosome": "8", 
    "gene_end": 144428563, 
    "canonical_transcript_id": "ENST00000377348", 
    "gene_strand": -1, 
    "gene_start": 144423601, 
    "name": "vacuolar protein sorting 28 homolog (S. cerevisiae)", 
    "gene_id": "ENSG00000160948"
  }, 
  "warnings": {}
}


### Example 5: Mutation Occurence information 

### Example 6: Survival data for cases with and without a given mutation 

# <a id='download'>How to Download Files with GDC API</a>

### Overview

- Users can download files by appending the file uuid to the end of the `data` endpoint url, i.e. `https://api.gdc.cancer.gov/data/5b2974ad-f932-499b-90a3-93577a9f0573`
- Can also create a manifest file that can be used to download files with the [GDC Data Transfer Tool](https://gdc.cancer.gov/access-data/gdc-data-transfer-tool)
- Documentation on downloading files using the GDC API can be found here: https://docs.gdc.cancer.gov/API/Users_Guide/Downloading_Files/

### Endpoint

- The endpoint to download files is `https://api.gdc.cancer.gov/data/`

### Steps



### Example 1: Downloading an open-access file

In [4]:
#url = "https://api.gdc.cancer.gov/data/<file_uuid>"

url = "https://api.gdc.cancer.gov/data/8690f7bd-de0c-41ee-806a-f4e949ed1fbf"

with requests.get(url, stream=True) as result:
    #with open("path/to/file_name.txt", "wb") as file:
    with open("test_download.txt", "wb") as file:
        for chunk in result.iter_content(chunk_size=16*1024):
            file.write(chunk)
file.close()

### Example 2: Downloading a controlled-access file

### Example 3: Related Files?

### Example 4: Downloading multiple files

# <a id='bam_slice'>How to Perform BAM Slicing with GDC API</a>

-  The GDC API provides remote BAM slicing functionality that enables downloading of specific parts of a BAM file instead of the whole file. 
- This functionality can be accessed at the `slicing` endpoint, using a syntax similar to that of widely used bioinformatics tools such as `samtools`
- BAM Slicing will require a token for authentication downloaded from the [GDC Data Portal](https://docs.gdc.cancer.gov/Data_Portal/Users_Guide/Cart/#gdc-authentication-tokens)
- Documentation for performing BAM slicing with the GDC API can be found here: https://docs.gdc.cancer.gov/API/Users_Guide/BAM_Slicing/

# <a id='submit'>How to Submit Data with GDC API</a>

### Overview

- Submitters can make use of the `submission` GDC API endpoint to submit node entities to submission projects
- Submission will require a token downloaded from the [GDC Submission Portal](https://docs.gdc.cancer.gov/Data_Submission_Portal/Users_Guide/Data_Submission_Process/#authentication)
- Data can be submitted in `JSON` or `TSV` format; depending on the data format, users will need to edit the `"Content-Type"` in the request command (see below)
- Submittable files (such as FASTQ or BAM files) should be uploaded with the [GDC Data Transfer Tool](https://gdc.cancer.gov/access-data/gdc-data-transfer-tool)
- Additional features and more information regarding submission using the GDC API can be found here: https://docs.gdc.cancer.gov/API/Users_Guide/Submission/ 
- [Strategies for Submitting in Bulk](https://docs.gdc.cancer.gov/Data_Submission_Portal/Users_Guide/Data_Submission_Walkthrough/#strategies-for-submitting-in-bulk)


### Endpoint

- The format for using the GDC API Submission enpoint uses the project information, i.e. `https://api.gdc.cancer.gov/<program_name>/<project_code>`
- for example: https://api.gdc.cancer.gov/submission/TCGA/LUAD or https://api.gdc.cancer.gov/submission/CPTAC/3 


### Steps

1. Read in token file
2. Read in submission file
3. Edit endpoint and submit data using `POST` request

### Example 1: Submitting a JSON Data File

In [ ]:
#1. Read in token file

token = open("path/to/file/gdc-user-token.txt").read().strip()

In [ ]:
#2. Read in submission file

example_file_json = json.load(open("example_file.json"))

In [ ]:
#3. Edit endpoint and submit data using POST request

ENDPT = "https://api.gdc.cancer.gov/submission/<program_name>/<project_id>"

#submission request if data is in JSON format
requests.post(url = ENDPT, json = example_file_json, headers={'X-Auth-Token': token, "Content-Type": "application/json"})

### Example 2: Submitting a TSV Data File

- Template TSV files can be found at the GDC Data Dictionary Viewer webpage: https://docs.gdc.cancer.gov/Data_Dictionary/viewer/#?_top=1

In [2]:
#1. Read in token file

token = open("path/to/file/gdc-user-token.txt").read().strip()

In [13]:
#2. Read in submission file

example_file_tsv = open("example_file.txt", "rb")

In [14]:
#3. Edit endpoint and submit data using PUT request

ENDPT = "https://api.gdc.cancer.gov/submission/<program_name>/<project_id>"

#submission request if data is in TSV format
res = requests.put(url = ENDPT, data = example_file_tsv, headers={'X-Auth-Token': token, "Content-Type": "text/tsv"})

res.text

'{"cases_related_to_created_entities_count":1,"cases_related_to_updated_entities_count":0,"code":200,"created_entity_count":2,"entities":[{"action":"create","errors":[],"id":"ba18283a-30ac-4ff8-b1b4-7d590792fd0c","related_cases":[{"id":"2c17941d-8488-4fdf-bb4d-2707a3b17312","submitter_id":"GDC-INTERNAL-000004"}],"type":"sample","unique_keys":[{"project_id":"GDC-INTERNAL","submitter_id":"Blood-00001_api1"}],"valid":true,"warnings":[]},{"action":"create","errors":[],"id":"f449697c-1271-4e50-94c7-567ecfb60e72","related_cases":[{"id":"2c17941d-8488-4fdf-bb4d-2707a3b17312","submitter_id":"GDC-INTERNAL-000004"}],"type":"sample","unique_keys":[{"project_id":"GDC-INTERNAL","submitter_id":"Blood-00001_api2"}],"valid":true,"warnings":[]}],"entity_error_count":0,"message":"Transaction successful.","success":true,"transaction_id":2674066,"transactional_error_count":0,"transactional_errors":[],"updated_entity_count":0}\n'